# Store Sales - Time Series Forecasting
1. 业务背景
+ 主要目标：该竞赛是使用时间序列预测来预测来自厄瓜多尔大型杂货零售商 Corporación Favorita 的数据的商店销售额。
+ 具体任务：利用**2013年1月1日至2017年8月15日**的厄瓜多尔地区不同商店不同商品类别不同日期的历史销售量，来预测**2017年8月16日至2017年8月31日**不同商品在指定日期的销售额。其中，时间序列预测就是利用过去一段时间的数据来预测未来一段时间内的信息。
+ 与销售额有关的因素
    + 政府：油价
    + 商家：促销活动
    + 用户：工资发放、节假日
    + 其他：地震

该任务是有监督学习（有训练数据、测试数据），由于是预测不同商品在指定日期的销售额，因此该学习任务是一个回归任务（给定商品和日期，输出销售额）。该任务的性能指标为均方根对数误差（Root Mean Squared Logarithmic Error，RMSLE）：

$$\sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2}$$

其中，$n$是实例的总数，$\hat{y}_i$是目标的预测值，$y_i$是目标的实际值，$\log$是自然对数（即以$e$为底）。

RMSLE 惩罚欠预测大于过预测，适用于某些需要欠预测损失更大的场景，在 RMSE 相同的情况下，预测值比真实值小这种情况的 RMSLE 比较大，即对于预测值小这种情况惩罚较大。

下面开始对数据进行分析：

In [99]:
import math
import numpy as np
import pandas as pd
import lightgbm as lgb

In [100]:
sales_train = pd.read_csv("./data/train.csv")   # 读取训练数据(2013年01月01日—2017年08月15日)

oil_data = pd.read_csv("./data/oil.csv")     # 读取石油价格数据(2013年01月01日—2017年08月31日)


In [101]:
# 查看训练数据基本信息
sales_train.head()
#sales_train.info()
#sales_train.describe()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


训练数据中共有3000888条数据，每条数据有6个特征：
+ id 表示商品唯一标识（int64）
+ date 表示商品的销售日期（object）
+ store_nbr 标识销售产品的商店（int64）
+ family 标识所售产品的类型（object）
+ sales 表示给定日期特定商店的产品系列的总销售额（float64）
+ onpromotion 表示在给定日期在商店促销的产品系列中的商品总数（int64）

比如数据(id=262935, date=2013/05/28, store_nbr=36, family=MEATS, sales=104.678, onpromotion=0)表示：
商品262935，在2013/05/28这一天，在商店36中，作为MEATS，销售额为104.678，而2013/05/28这一天商店促销的MEATS的商品总数为0

In [102]:
# 由于上述训练数据比较完好，因此直接进行线下验证
# 将2017年08月01日—2017年08月15日的训练数据作为验证数据，其余2013年01月01日—2017年07月31日作为训练数据

train_set = sales_train.loc[("2013-01-01" <= sales_train["date"]) & (sales_train["date"] <= "2017-07-31")]
valid_set = sales_train.loc[("2017-08-01" <= sales_train["date"]) & (sales_train["date"] <= "2017-08-15")]

In [103]:
# 将训练数据和验证数据中的非数值属性转换为数值属性
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
def numeric_transform(data):
    data_date = data[["date"]]
    data[['date']] = ordinal_encoder.fit_transform(data_date)
    data_family = data[["family"]]
    data[['family']] = ordinal_encoder.fit_transform(data_family)

numeric_transform(train_set)
numeric_transform(valid_set)
train_set.head()

/tmp/ipykernel_427915/514964345.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['date']] = ordinal_encoder.fit_transform(data_date)
/tmp/ipykernel_427915/514964345.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['family']] = ordinal_encoder.fit_transform(data_family)
/tmp/ipykernel_427915/514964345.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,id,date,store_nbr,family,sales,onpromotion
0,0,0.0,1,0.0,0.0,0
1,1,0.0,1,1.0,0.0,0
2,2,0.0,1,2.0,0.0,0
3,3,0.0,1,3.0,0.0,0
4,4,0.0,1,4.0,0.0,0


In [104]:
# 将训练数据和验证数据划分为特征和标签

# 训练数据
train_set_x = train_set[['id','date','store_nbr','family','onpromotion']]
train_set_y = train_set[["sales"]]

# 验证数据
valid_set_x = valid_set[['id','date','store_nbr','family','onpromotion']]
valid_set_y = valid_set[["sales"]]
valid_set_x.index = range(len(valid_set_x))
valid_set_y.index = range(len(valid_set_y))

In [105]:
# baseline方案：LightGBM模型训练

# 定义损失函数
def loss_rmsle(y_pred, y_true):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    return np.sqrt(np.average(np.square(np.log(y_pred+1), np.log(y_true+1))))


train_set_lgb = lgb.Dataset(train_set_x, train_set_y)
valid_set_lgb = lgb.Dataset(valid_set_x, valid_set_y)

params = {'objective':'regression',  # 目标函数
          'learning_rate':0.01,
          'boost_type':'gbdt',       # 设置提升类型
          'seed' : 2022,
          'metric' : 'rmse'}

lgb_reg = lgb.train(params,
                    train_set = train_set_lgb,
                    num_boost_round = 2000,
                    valid_sets = valid_set_lgb,
                    early_stopping_rounds = 100)


/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: boost_type
[LightGBM] [Warning] Unknown parameter: boost_type
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009166 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 802
[LightGBM] [Info] Number of data points in the train set: 2974158, number of used features: 5
[LightGBM] [Warning] Unknown parameter: boost_type
[LightGBM] [Info] Start training from score 356.810778
[1]	valid_0's rmse: 1240.26
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 1231.65
[3]	valid_0's rmse: 1223.09
[4]	valid_0's rmse: 1214.7
[5]	valid_0's rmse: 1206.16
[6]	valid_0's rmse: 1197.91
[7]	valid_0's rmse: 1189.75
[8]	valid_0's rmse: 1181.5
[9]	valid_0's rmse: 1173.36
[10]	valid_0's rmse: 1165.57
[11]	valid_0's rmse: 1157.86
[12]	valid_0's rmse: 1149.99
[13]	valid_0's rmse: 1142.41
[14]	val

In [106]:
# 模型预测
sales_test = pd.read_csv("./data/test.csv")     # 读取测试数据(2017年08月16日—2017年08月31日)
numeric_transform(sales_test)

sample_submission_predict = pd.read_csv("./data/sample_submission.csv")  # 读取提交数据表
sample_submission_predict = sample_submission_predict[['id', 'sales']]

sample_submission_predict_sales = lgb_reg.predict(sales_test)             # 数据预测
sample_submission_predict[['sales']] = sample_submission_predict_sales[:,np.newaxis]

sample_submission_predict.sales[sample_submission_predict.sales <= 0] = 0 # 提交数据表后处理

sample_submission_predict



/tmp/ipykernel_427915/2479762615.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_submission_predict.sales[sample_submission_predict.sales <= 0] = 0 # 提交数据表后处理


,id,sales
0,3000888,0.000000
1,3000889,0.000000
2,3000890,0.000000
3,3000891,2445.377486
4,3000892,0.000000
...,...,...
28507,3029395,425.955369
28508,3029396,178.006840
28509,3029397,886.135934
28510,3029398,225.445396


In [107]:
sample_submission_predict.to_csv("./data/sample_submission.csv")